In [ ]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility: load or init JSON file
def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient Management
def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    contact = input('Contact details: ').strip()
    record = {'id': new_id, 'name': name, 'dob': dob, 'contact': contact}
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')

def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}, DOB: {p['dob']}, Contact: {p['contact']}")

def search_patient():
    term = input('Search by name or ID: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in str(p['id']) or term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}, DOB: {p['dob']}, Contact: {p['contact']}")

# SOAP Note Input (SOAP: Subjective, Objective, Action, Plan)
def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')

# View Patient Notes
def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(
            f"Date: {n['date']}\n"
            f"  S: {n['subjective']}\n"
            f"  O: {n['objective']}\n"
            f"  A: {n['action']}\n"
            f"  P: {n['plan']}\n"
        )

# Word Report Generation
def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()
    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return
    entries = [n for n in notes if n['patient_id'] == patient_id and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])
    doc = Document()
    doc.add_heading('Sample Physiotherapy Progress Report', level=0)
    doc.add_heading('Patient Information:', level=1)
    doc.add_paragraph(f"Patient Name: {patient['name']}")
    doc.add_paragraph(f"Patient ID: {patient['id']}")
    doc.add_paragraph(f"Date of Birth: {patient['dob']}")
    doc.add_paragraph("Referring Physician (if applicable): ")
    doc.add_paragraph(f"Date of Initial Assessment: {entries[0]['date']}")
    report_date = datetime.now().date().isoformat()
    doc.add_paragraph(f"Report Date: {report_date}")
    doc.add_paragraph("Reason for Referral/Chief Complaint: ")
    doc.add_heading('Current Condition (Summary based on recent SOAP notes):', level=1)
    latest = entries[-1]
    doc.add_paragraph(f"Subjective: {latest['subjective']}")
    doc.add_paragraph(f"Objective: {latest['objective']}")
    doc.add_paragraph(f"Assessment: {latest['action']}")
    doc.add_paragraph(f"Plan: {latest['plan']}")
    doc.add_heading('Progress Since Last Report (if applicable):', level=1)
    if len(entries) > 1:
        for n in entries[1:]:
            doc.add_paragraph(f"{n['date']}: Assessment - {n['action']}")
    else:
        doc.add_paragraph('N/A')
    doc.add_heading('Goals:', level=1)
    doc.add_paragraph('Short-Term Goals (as of this report): ')
    doc.add_paragraph('Progress Towards Short-Term Goals: ')
    doc.add_paragraph('Long-Term Goals (overall objectives of therapy): ')
    doc.add_heading('Treatment Interventions Utilized:', level=1)
    doc.add_paragraph('')
    doc.add_heading('Patient Education and Compliance:', level=1)
    doc.add_paragraph('')
    doc.add_heading('Factors Affecting Progress:', level=1)
    doc.add_paragraph('')
    doc.add_heading('Recommendations:', level=1)
    doc.add_paragraph('')
    doc.add_heading('Prognosis:', level=1)
    doc.add_paragraph('')
    doc.add_paragraph('Therapist Information:')
    doc.add_paragraph('Therapist Name: ')
    doc.add_paragraph('Signature: ')
    doc.add_paragraph('Date: ')
    filename = f"Progress_Report_patient{patient_id}_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI Menu
def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


In [2]:
!pip install python-docx

In [2]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility: load or init JSON file
def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient Management
def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    contact = input('Contact details: ').strip()
    record = {'id': new_id, 'name': name, 'dob': dob, 'contact': contact}
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')

def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}, DOB: {p['dob']}, Contact: {p['contact']}")

def search_patient():
    term = input('Search by name or ID: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in str(p['id']) or term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}, DOB: {p['dob']}, Contact: {p['contact']}")

# SOAP Note Input (SOAP: Subjective, Objective, Action, Plan)
def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')

# View Patient Notes
def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(
            f"Date: {n['date']}\n"
            f"  S: {n['subjective']}\n"
            f"  O: {n['objective']}\n"
            f"  A: {n['action']}\n"
            f"  P: {n['plan']}\n"
        )

# Word Report Generation
def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()
    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return
    entries = [n for n in notes if n['patient_id'] == patient_id and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])
    doc = Document()
    doc.add_heading('Sample Physiotherapy Progress Report', level=0)
    doc.add_heading('Patient Information:', level=1)
    doc.add_paragraph(f"Patient Name: {patient['name']}")
    doc.add_paragraph(f"Patient ID: {patient['id']}")
    doc.add_paragraph(f"Date of Birth: {patient['dob']}")
    doc.add_paragraph("Referring Physician (if applicable): ")
    doc.add_paragraph(f"Date of Initial Assessment: {entries[0]['date']}")
    report_date = datetime.now().date().isoformat()
    doc.add_paragraph(f"Report Date: {report_date}")
    doc.add_paragraph("Reason for Referral/Chief Complaint: ")
    doc.add_heading('Current Condition (Summary based on recent SOAP notes):', level=1)
    latest = entries[-1]
    doc.add_paragraph(f"Subjective: {latest['subjective']}")
    doc.add_paragraph(f"Objective: {latest['objective']}")
    doc.add_paragraph(f"Assessment: {latest['action']}")
    doc.add_paragraph(f"Plan: {latest['plan']}")
    doc.add_heading('Progress Since Last Report (if applicable):', level=1)
    if len(entries) > 1:
        for n in entries[1:]:
            doc.add_paragraph(f"{n['date']}: Assessment - {n['action']}")
    else:
        doc.add_paragraph('N/A')
    doc.add_heading('Goals:', level=1)
    doc.add_paragraph('Short-Term Goals (as of this report): ')
    doc.add_paragraph('Progress Towards Short-Term Goals: ')
    doc.add_paragraph('Long-Term Goals (overall objectives of therapy): ')
    doc.add_heading('Treatment Interventions Utilized:', level=1)
    doc.add_paragraph('')
    doc.add_heading('Patient Education and Compliance:', level=1)
    doc.add_paragraph('')
    doc.add_heading('Factors Affecting Progress:', level=1)
    doc.add_paragraph('')
    doc.add_heading('Recommendations:', level=1)
    doc.add_paragraph('')
    doc.add_heading('Prognosis:', level=1)
    doc.add_paragraph('')
    doc.add_paragraph('Therapist Information:')
    doc.add_paragraph('Therapist Name: ')
    doc.add_paragraph('Signature: ')
    doc.add_paragraph('Date: ')
    filename = f"Progress_Report_patient{patient_id}_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI Menu
def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  2


ID: 1, Name: Michael, DOB: 19990-08-31, Contact: 0814879874
ID: 2, Name: Tomas, DOB: 2000-11-20, Contact: 0812228888

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  6
Patient ID for report:  2
Start date (YYYY-MM-DD):  2025-08-22
End date (YYYY-MM-DD):  2025-08-22


No notes in given range.

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  6
Patient ID for report:  1
Start date (YYYY-MM-DD):  2025-04-22
End date (YYYY-MM-DD):  2025-04-24


Report saved to Progress_Report_patient1_2025-04-22_to_2025-04-24.docx

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  1
Patient name:  0
Date of birth (YYYY-MM-DD):  we
Contact details:  qwr


Added patient ID 3

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  0


In [4]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility: load or init JSON file
def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient Management
def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    contact = input('Contact details: ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'contact': contact,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')

def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(
            f"ID: {p['id']}, Name: {p['name']}, DOB: {p['dob']}, Contact: {p['contact']}, "
            f"Medical Aid: {p.get('medical_aid','')}, Medical Aid Number: {p.get('medical_aid_number','')}"
        )

def search_patient():
    term = input('Search by name or ID: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in str(p['id']) or term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(
            f"ID: {p['id']}, Name: {p['name']}, DOB: {p['dob']}, Contact: {p['contact']}, "
            f"Medical Aid: {p.get('medical_aid','')}, Medical Aid Number: {p.get('medical_aid_number','')}"
        )

# SOAP Note Input (SOAP: Subjective, Objective, Action, Plan)
def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')

# View Patient Notes
def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(
            f"Date: {n['date']}\n"
            f"  S: {n['subjective']}\n"
            f"  O: {n['objective']}\n"
            f"  A: {n['action']}\n"
            f"  P: {n['plan']}\n"
        )

# Word Report Generation
def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    entries = [n for n in notes 
               if n['patient_id'] == patient_id 
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    doc = Document()
    doc.add_heading('Sample Physiotherapy Progress Report', level=0)

    # Patient Information
    doc.add_heading('Patient Information:', level=1)
    doc.add_paragraph(f"Patient Name: {patient['name']}")
    doc.add_paragraph(f"Patient ID: {patient['id']}")
    doc.add_paragraph(f"Date of Birth: {patient['dob']}")
    doc.add_paragraph(f"Medical Aid: {patient.get('medical_aid','')}")
    doc.add_paragraph(f"Medical Aid Number: {patient.get('medical_aid_number','')}")
    doc.add_paragraph("Referring Physician (if applicable): ")
    doc.add_paragraph(f"Date of Initial Assessment: {entries[0]['date']}")
    report_date = datetime.now().date().isoformat()
    doc.add_paragraph(f"Report Date: {report_date}")
    doc.add_paragraph("Reason for Referral/Chief Complaint: ")

    # Current Condition
    doc.add_heading('Current Condition Summary:', level=1)
    latest = entries[-1]
    doc.add_paragraph(f"Subjective: {latest['subjective']}")
    doc.add_paragraph(f"Objective: {latest['objective']}")

    # Action subsection
    doc.add_heading('Action:', level=1)
    doc.add_paragraph(f"{latest['action']}")
    doc.add_paragraph('Progress Since Last Report: ')
    doc.add_paragraph('Treatment Interventions Utilized: ')
    doc.add_paragraph('Patient Education and Compliance: ')
    doc.add_paragraph('Factors Affecting Progress: ')

    # Plan subsection
    doc.add_heading('Plan:', level=1)
    doc.add_paragraph(f"{latest['plan']}")
    doc.add_paragraph('Goals: ')
    doc.add_paragraph('Recommendations: ')
    doc.add_paragraph('Prognosis: ')

    # Therapist Information
    doc.add_paragraph('Therapist Information:')
    doc.add_paragraph('Therapist Name: ')
    doc.add_paragraph('Signature: ')
    doc.add_paragraph('Date: ')

    filename = f"Progress_Report_patient{patient_id}_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI Menu
def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  1
Patient name:  Randif
Date of birth (YYYY-MM-DD):  1990-08-21
Contact details:  0814227777
Medical Aid:  NMC
Medical Aid Number:  4329857


Added patient ID 4

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  4
Patient ID:  4
Session date (YYYY-MM-DD):  2025-02-25
Subjective:  He said he was tired
Objective:  Gr 2 fatigue
Action:  Gave him some exercise
Plan:  Continue exercise


SOAP note saved.

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


...

Select an option:  5
Patient ID to view notes:  4


Date: 2025-02-25
  S: He said he was tired
  O: Gr 2 fatigue
  A: Gave him some exercise
  P: Continue exercise

Date: 2025-03-10
  S: He said he's okay
  O: Gr 5 okayness
  A: Kept him ok
  P: Make him better

Date: 2025-04-25
  S: Bagged
  O: Looked dead
  A: Woke him up
  P: Send him off


=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  6
Patient ID for report:  4
Start date (YYYY-MM-DD):  2025-01-01
End date (YYYY-MM-DD):  2025-03-31


Report saved to Progress_Report_patient4_2025-01-01_to_2025-03-31.docx

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  0


In [6]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility: load or init JSON file
def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient Management
def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM‑DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')

def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")

def search_patient():
    term = input('Search by name or ID: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in str(p['id']) or term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP Note Input
def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM‑DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')

def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(
            f"Date: {n['date']}\n"
            f"  S: {n['subjective']}\n"
            f"  O: {n['objective']}\n"
            f"  A: {n['action']}\n"
            f"  P: {n['plan']}\n"
        )

# Word Report Generation
def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM‑DD): ').strip()
    end = input('End date (YYYY-MM‑DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    # Create document
    doc = Document()
    # Default font
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)

    # Title
    doc.add_heading(f"{patient['name']} Report", level=0)

    # Patient information table (2 columns)
    table = doc.add_table(rows=4, cols=2)
    # Row 1
    row = table.rows[0].cells
    row[0].text = f"Name: {patient['name']}"
    row[1].text = f"Date of Birth: {patient['dob']}"
    # Row 2
    row = table.rows[1].cells
    row[0].text = f"Medical Aid: {patient['medical_aid']}"
    row[1].text = f"Medical Aid Number: {patient['medical_aid_number']}"
    # Row 3
    row = table.rows[2].cells
    row[0].text = f"Occupation: {patient['occupation']}"
    row[1].text = f"Physiotherapist: {patient['physiotherapist']}"
    # Row 4 (Referral Diagnosis spans both)
    row = table.rows[3].cells
    row[0].text = f"Referral Diagnosis: {patient['diagnosis']}"
    row[0].merge(row[1])

    doc.add_paragraph()  # spacing

    # Paragraph 1: Subjective and Objective of first session
    first = entries[0]
    p1 = f"{first['subjective']}. {first['objective']}."
    doc.add_paragraph(p1)

    # Paragraph 2: Management (Action) of each session
    actions = [f"On {n['date']}, {n['action']}." for n in entries]
    doc.add_paragraph(" ".join(actions))

    # Paragraph 3: Plans/Recommendations
    plans = [f"On {n['date']}, {n['plan']}." for n in entries]
    doc.add_paragraph(" ".join(plans))

    # Save
    safe_name = patient['name'].replace(' ', '_')
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI Menu
def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  5
Patient ID to view notes:  4


Date: 2025-02-25
  S: He said he was tired
  O: Gr 2 fatigue
  A: Gave him some exercise
  P: Continue exercise

Date: 2025-03-10
  S: He said he's okay
  O: Gr 5 okayness
  A: Kept him ok
  P: Make him better

Date: 2025-04-25
  S: Bagged
  O: Looked dead
  A: Woke him up
  P: Send him off


=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  6
Patient ID for report:  4
Start date (YYYY-MM‑DD):  2025-01-01
End date (YYYY-MM‑DD):  2025-03-31


KeyError: 'occupation'

In [8]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient Management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')


def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")


def search_patient():
    term = input('Search by name or ID: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in str(p['id']) or term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP Note Input

def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')


def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(
            f"Date: {n['date']}\n"
            f"  S: {n['subjective']}\n"
            f"  O: {n['objective']}\n"
            f"  A: {n['action']}\n"
            f"  P: {n['plan']}\n"
        )

# Word Report Generation

def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    doc = Document()
    # Default font
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)

    # Title
    doc.add_heading(f"{patient.get('name','')} Report", level=0)

    # Patient information table
    table = doc.add_table(rows=4, cols=2)
    # Row 1
    row = table.rows[0].cells
    row[0].text = f"Name: {patient.get('name','')}"
    row[1].text = f"Date of Birth: {patient.get('dob','')}"
    # Row 2
    row = table.rows[1].cells
    row[0].text = f"Medical Aid: {patient.get('medical_aid','')}"
    row[1].text = f"Medical Aid Number: {patient.get('medical_aid_number','')}"
    # Row 3
    row = table.rows[2].cells
    row[0].text = f"Occupation: {patient.get('occupation','')}"
    row[1].text = f"Physiotherapist: {patient.get('physiotherapist','')}"
    # Row 4
    row = table.rows[3].cells
    cell = row[0]
    cell.text = f"Referral Diagnosis: {patient.get('diagnosis','')}"
    cell.merge(row[1])

    doc.add_paragraph()

    # Paragraph 1: first session subjective/objective
    first = entries[0]
    p1 = f"{first.get('subjective','')} {first.get('objective','')}"
    doc.add_paragraph(p1)

    # Paragraph 2: actions of each session
    actions = [f"On {n.get('date','')}, {n.get('action','')}" for n in entries]
    doc.add_paragraph('. '.join(actions) + '.')

    # Paragraph 3: plans of each session
    plans = [f"On {n.get('date','')}, {n.get('plan','')}" for n in entries]
    doc.add_paragraph('. '.join(plans) + '.')

    # Save document
    safe_name = patient.get('name','').replace(' ', '_') or 'Patient'
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI Menu

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()


=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  6
Patient ID for report:  4
Start date (YYYY-MM-DD):  2025-01-01
End date (YYYY-MM-DD):  2025-03-31


Report saved to Randif_Report_2025-01-01_to_2025-03-31.docx

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  0


In [10]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')


def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")


def search_patient():
    term = input('Search by name or ID: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in str(p['id']) or term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP note input

def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')


def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(f"Date: {n['date']}\n  S: {n['subjective']}\n  O: {n['objective']}\n  A: {n['action']}\n  P: {n['plan']}\n")

# Report generation

def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    # filter notes in range
    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    doc = Document()
    # set default font
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)

    # title
    doc.add_heading(f"{patient.get('name','')} Report", level=0)

    # patient info table (4 rows x 2 cols)
    table = doc.add_table(rows=4, cols=2)
    # row 1
    row = table.rows[0].cells
    row[0].text = f"Patient Name: {patient.get('name','')}"
    row[1].text = f"Patient ID: {patient.get('id','')}"
    # row 2
    row = table.rows[1].cells
    row[0].text = f"Date of Birth: {patient.get('dob','')}"
    row[1].text = f"Date of Initial Assessment: {entries[0].get('date','')}"
    # row 3
    row = table.rows[2].cells
    report_date = datetime.now().date().isoformat()
    row[0].text = f"Report Date: {report_date}"
    row[1].text = f"Reason for Referral/Chief Complaint: {patient.get('diagnosis','')}"
    # row 4
    row = table.rows[3].cells
    row[0].text = f"Occupation: {patient.get('occupation','')}"
    row[1].text = f"Physiotherapist: {patient.get('physiotherapist','')}"

    doc.add_paragraph()

    # paragraph 1: subjective + objective of first session
    first = entries[0]
    para1 = f"{first.get('subjective','')} {first.get('objective','')}"
    doc.add_paragraph(para1)

    # paragraph 2: all actions without dates
    actions = [n.get('action','') for n in entries]
    doc.add_paragraph('. '.join(actions) + '.')

    # paragraph 3: all plans without dates
    plans = [n.get('plan','') for n in entries]
    doc.add_paragraph('. '.join(plans) + '.')

    # save
    safe_name = patient.get('name','').replace(' ', '_') or 'Patient'
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()


=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  6
Patient ID for report:  4
Start date (YYYY-MM-DD):  2025-01-01
End date (YYYY-MM-DD):  2025-03-31


Report saved to Randif_Report_2025-01-01_to_2025-03-31.docx

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  0


In [12]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')


def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")


def search_patient():
    term = input('Search by name or ID: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in str(p['id']) or term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP note input

def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')


def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(f"Date: {n['date']}\n  S: {n['subjective']}\n  O: {n['objective']}\n  A: {n['action']}\n  P: {n['plan']}\n")

# Report generation

def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    # filter notes in range
    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    doc = Document()
    # set default font
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)

    # title
    doc.add_heading(f"{patient.get('name','')} Report", level=0)

    # patient info table (5 rows x 2 cols)
    table = doc.add_table(rows=5, cols=2)
    # row 1
    row = table.rows[0].cells
    row[0].text = f"Patient Name: {patient.get('name','')}"
    row[1].text = f"Patient ID: {patient.get('id','')}"
    # row 2
    row = table.rows[1].cells
    row[0].text = f"Date of Birth: {patient.get('dob','')}"
    row[1].text = f"Date of Initial Assessment: {entries[0].get('date','')}"
    # row 3
    row = table.rows[2].cells
    report_date = datetime.now().date().isoformat()
    row[0].text = f"Report Date: {report_date}"
    row[1].text = f"Reason for Referral/Chief Complaint: {patient.get('diagnosis','')}"
    # row 4
    row = table.rows[3].cells
    row[0].text = f"Medical Aid: {patient.get('medical_aid','')}"
    row[1].text = f"Medical Aid Number: {patient.get('medical_aid_number','')}"
    # row 5
    row = table.rows[4].cells
    row[0].text = f"Occupation: {patient.get('occupation','')}"
    row[1].text = f"Physiotherapist: {patient.get('physiotherapist','')}"

    doc.add_paragraph()

    # paragraph 1: subjective + objective of first session
    first = entries[0]
    para1 = f"{first.get('subjective','')} {first.get('objective','')}"
    doc.add_paragraph(para1)

    # paragraph 2: all actions without dates
    actions = [n.get('action','') for n in entries]
    doc.add_paragraph('. '.join(actions) + '.')

    # paragraph 3: all plans without dates
    plans = [n.get('plan','') for n in entries]
    doc.add_paragraph('. '.join(plans) + '.')

    # save
    safe_name = patient.get('name','').replace(' ', '_') or 'Patient'
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  6
Patient ID for report:  4
Start date (YYYY-MM-DD):  2025-01-01
End date (YYYY-MM-DD):  2025-03-31


Report saved to Randif_Report_2025-01-01_to_2025-03-31.docx

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  0


In [ ]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')


def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")


def search_patient():
    term = input('Search by name or ID: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in str(p['id']) or term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP note input

def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')


def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(f"Date: {n['date']}\n  S: {n['subjective']}\n  O: {n['objective']}\n  A: {n['action']}\n  P: {n['plan']}\n")

# Report generation

def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    doc = Document()
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)

    # patient info table (5 rows x 2 cols)
    table = doc.add_table(rows=5, cols=2)
    row = table.rows[0].cells
    row[0].text = f"Patient Name: {patient.get('name','')}"
    row[1].text = f"Patient ID: {patient.get('id','')}"
    row = table.rows[1].cells
    row[0].text = f"Date of Birth: {patient.get('dob','')}"
    row[1].text = f"Date of Initial Assessment: {entries[0].get('date','')}"
    row = table.rows[2].cells
    row[0].text = f"Medical Aid: {patient.get('medical_aid','')}"
    row[1].text = f"Medical Aid Number: {patient.get('medical_aid_number','')}"
    row = table.rows[3].cells
    row[0].text = f"Occupation: {patient.get('occupation','')}"
    row[1].text = f"Physiotherapist: {patient.get('physiotherapist','')}"
    row = table.rows[4].cells
    report_date = datetime.now().date().isoformat()
    row[0].text = f"Report Date: {report_date}"
    row[1].text = f"Reason for Referral/Chief Complaint: {patient.get('diagnosis','')}"

    doc.add_paragraph()
    first = entries[0]
    para1 = f"{first.get('subjective','')} {first.get('objective','')}"
    doc.add_paragraph(para1)
    actions = [n.get('action','') for n in entries]
    doc.add_paragraph('. '.join(actions) + '.')
    plans = [n.get('plan','') for n in entries]
    doc.add_paragraph('. '.join(plans) + '.')

    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('Regards')
    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('______________________________')
    doc.add_paragraph('Ronelle Isaacs Physiotherapist')

    safe_name = patient.get('name','').replace(' ', '_') or 'Patient'
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  6
Patient ID for report:  4
Start date (YYYY-MM-DD):  2025-01-01
End date (YYYY-MM-DD):  2025-03-31


Report saved to Randif_Report_2025-01-01_to_2025-03-31.docx

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


In [ ]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')


def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")


def search_patient():
    term = input('Search by name: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP note input

def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')


def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(f"Date: {n['date']}\n  S: {n['subjective']}\n  O: {n['objective']}\n  A: {n['action']}\n  P: {n['plan']}\n")

# Report generation

def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    doc = Document()
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)

    # patient info table (5 rows x 2 cols)
    table = doc.add_table(rows=5, cols=2)
    row = table.rows[0].cells
    row[0].text = f"Patient Name: {patient.get('name','')}"
    row[1].text = f"Patient ID: {patient.get('id','')}"
    row = table.rows[1].cells
    row[0].text = f"Date of Birth: {patient.get('dob','')}"
    row[1].text = f"Date of Initial Assessment: {entries[0].get('date','')}"
    row = table.rows[2].cells
    row[0].text = f"Medical Aid: {patient.get('medical_aid','')}"
    row[1].text = f"Medical Aid Number: {patient.get('medical_aid_number','')}"
    row = table.rows[3].cells
    row[0].text = f"Occupation: {patient.get('occupation','')}"
    row[1].text = f"Physiotherapist: {patient.get('physiotherapist','')}"
    row = table.rows[4].cells
    report_date = datetime.now().date().isoformat()
    row[0].text = f"Report Date: {report_date}"
    row[1].text = f"Reason for Referral/Chief Complaint: {patient.get('diagnosis','')}"

    doc.add_paragraph()
    first = entries[0]
    para1 = f"{first.get('subjective','')} {first.get('objective','')}"
    doc.add_paragraph(para1)
    actions = [n.get('action','') for n in entries]
    doc.add_paragraph('. '.join(actions) + '.')
    plans = [n.get('plan','') for n in entries]
    doc.add_paragraph('. '.join(plans) + '.')

    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('Regards')
    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('______________________________')
    doc.add_paragraph('Ronelle Isaacs Physiotherapist')

    safe_name = patient.get('name','').replace(' ', '_') or 'Patient'
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients by name', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()


In [ ]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')


def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")


def search_patient():
    term = input('Search by name: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP note input

def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')


def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(f"Date: {n['date']}\n  S: {n['subjective']}\n  O: {n['objective']}\n  A: {n['action']}\n  P: {n['plan']}\n")

# Report generation

def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    doc = Document()
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)

    # patient info table (5 rows x 2 cols)
    table = doc.add_table(rows=5, cols=2)
    row = table.rows[0].cells
    row[0].text = f"Patient Name: {patient.get('name','')}"
    row[1].text = f"Patient ID: {patient.get('id','')}"
    row = table.rows[1].cells
    row[0].text = f"Date of Birth: {patient.get('dob','')}"
    row[1].text = f"Date of Initial Assessment: {entries[0].get('date','')}"
    row = table.rows[2].cells
    row[0].text = f"Medical Aid: {patient.get('medical_aid','')}"
    row[1].text = f"Medical Aid Number: {patient.get('medical_aid_number','')}"
    row = table.rows[3].cells
    row[0].text = f"Occupation: {patient.get('occupation','')}"
    row[1].text = f"Physiotherapist: {patient.get('physiotherapist','')}"
    row = table.rows[4].cells
    report_date = datetime.now().date().isoformat()
    row[0].text = f"Report Date: {report_date}"
    row[1].text = f"Reason for Referral/Chief Complaint: {patient.get('diagnosis','')}"

    doc.add_paragraph()
    first = entries[0]
    para1 = f"{first.get('subjective','')} {first.get('objective','')}"
    doc.add_paragraph(para1)
    actions = [n.get('action','') for n in entries]
    doc.add_paragraph('. '.join(actions) + '.')
    plans = [n.get('plan','') for n in entries]
    doc.add_paragraph('. '.join(plans) + '.')

    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('Regards')
    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('______________________________')
    doc.add_paragraph('Ronelle Isaacs Physiotherapist')

    safe_name = patient.get('name','').replace(' ', '_') or 'Patient'
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients by name', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()


In [ ]:
import json
import os
import sys
from datetime import datetime

# Ensure python-docx is installed
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    print("Error: python-docx not found. Install with 'pip install python-docx'")
    sys.exit(1)

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')


def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")


def search_patient():
    term = input('Search by name: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP note input

def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')


def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(f"Date: {n['date']}\n  S: {n['subjective']}\n  O: {n['objective']}\n  A: {n['action']}\n  P: {n['plan']}\n")

# Report generation

def generate_progress_report():
    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    doc = Document()
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)

    # patient info table (5 rows x 2 cols)
    table = doc.add_table(rows=5, cols=2)
    row = table.rows[0].cells
    row[0].text = f"Patient Name: {patient.get('name','')}"
    row[1].text = f"Patient ID: {patient.get('id','')}"
    row = table.rows[1].cells
    row[0].text = f"Date of Birth: {patient.get('dob','')}"
    row[1].text = f"Date of Initial Assessment: {entries[0].get('date','')}"
    row = table.rows[2].cells
    row[0].text = f"Medical Aid: {patient.get('medical_aid','')}"
    row[1].text = f"Medical Aid Number: {patient.get('medical_aid_number','')}"
    row = table.rows[3].cells
    row[0].text = f"Occupation: {patient.get('occupation','')}"
    row[1].text = f"Physiotherapist: {patient.get('physiotherapist','')}"
    row = table.rows[4].cells
    report_date = datetime.now().date().isoformat()
    row[0].text = f"Report Date: {report_date}"
    row[1].text = f"Reason for Referral/Chief Complaint: {patient.get('diagnosis','')}"

    doc.add_paragraph()
    first = entries[0]
    para1 = f"{first.get('subjective','')} {first.get('objective','')}"
    doc.add_paragraph(para1)
    actions = [n.get('action','') for n in entries]
    doc.add_paragraph('. '.join(actions) + '.')
    plans = [n.get('plan','') for n in entries]
    doc.add_paragraph('. '.join(plans) + '.')

    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('Regards')
    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('______________________________')
    doc.add_paragraph('Ronelle Isaacs Physiotherapist')

    safe_name = patient.get('name','').replace(' ', '_') or 'Patient'
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients by name', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()


In [ ]:
import json
import os
import sys
from datetime import datetime

# Attempt to import python-docx, but do not exit if missing
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    Document = None
    Pt = None
    print("Warning: python-docx not found. Report generation will be disabled.")

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')


def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")


def search_patient():
    term = input('Search by name: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP note input

def add_soap_note():
    notes = _load_data(NOTES_FILE)
    patient_id = int(input('Patient ID: ').strip())
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')


def view_soap_notes():
    patient_id = int(input('Patient ID to view notes: ').strip())
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(f"Date: {n['date']}\n  S: {n['subjective']}\n  O: {n['objective']}\n  A: {n['action']}\n  P: {n['plan']}\n")

# Report generation

def generate_progress_report():
    if Document is None:
        print("Cannot generate report: python-docx missing.")
        return

    patient_id = int(input('Patient ID for report: ').strip())
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()

    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return

    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])

    doc = Document()
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)

    # patient info table (5 rows x 2 cols)
    table = doc.add_table(rows=5, cols=2)
    row = table.rows[0].cells
    row[0].text = f"Patient Name: {patient.get('name','')}"
    row[1].text = f"Patient ID: {patient.get('id','')}"
    row = table.rows[1].cells
    row[0].text = f"Date of Birth: {patient.get('dob','')}"
    row[1].text = f"Date of Initial Assessment: {entries[0].get('date','')}"
    row = table.rows[2].cells
    row[0].text = f"Medical Aid: {patient.get('medical_aid','')}"
    row[1].text = f"Medical Aid Number: {patient.get('medical_aid_number','')}"
    row = table.rows[3].cells
    row[0].text = f"Occupation: {patient.get('occupation','')}"
    row[1].text = f"Physiotherapist: {patient.get('physiotherapist','')}"
    row = table.rows[4].cells
    report_date = datetime.now().date().isoformat()
    row[0].text = f"Report Date: {report_date}"
    row[1].text = f"Reason for Referral/Chief Complaint: {patient.get('diagnosis','')}"

    doc.add_paragraph()
    first = entries[0]
    para1 = f"{first.get('subjective','')} {first.get('objective','')}"
    doc.add_paragraph(para1)
    actions = [n.get('action','') for n in entries]
    doc.add_paragraph('. '.join(actions) + '.')
    plans = [n.get('plan','') for n in entries]
    doc.add_paragraph('. '.join(plans) + '.')

    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('Regards')
    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('______________________________')
    doc.add_paragraph('Ronelle Isaacs Physiotherapist')

    safe_name = patient.get('name','').replace(' ', '_') or 'Patient'
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients by name', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()

In [ ]:
import json
import os
import sys
from datetime import datetime

# Attempt to import python-docx, but do not exit if missing
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    Document = None
    Pt = None
    print("Warning: python-docx not found. Report generation will be disabled.")

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Helper to select patient by name

def select_patient(prompt="Patient name: "):
    patients = _load_data(PATIENTS_FILE)
    term = input(prompt).strip().lower()
    matches = [p for p in patients if term in p['name'].lower()]
    if not matches:
        print('No matches found.')
        return None
    if len(matches) == 1:
        p = matches[0]
        print(f"Selected: ID {p['id']}, Name: {p['name']}")
        return p['id']
    print('Multiple matches found:')
    for i, p in enumerate(matches, start=1):
        print(f"{i}. ID: {p['id']}, Name: {p['name']}")
    sel = input('Select patient number: ').strip()
    try:
        idx = int(sel) - 1
        chosen = matches[idx]
        return chosen['id']
    except:
        print('Invalid selection.')
        return None

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1
    name = input('Patient name: ').strip()
    dob = input('Date of birth (YYYY-MM-DD): ').strip()
    medical_aid = input('Medical Aid: ').strip()
    medical_aid_number = input('Medical Aid Number: ').strip()
    occupation = input('Occupation: ').strip()
    physiotherapist = input('Physiotherapist: ').strip()
    diagnosis = input('Referral Diagnosis: ').strip()
    record = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'occupation': occupation,
        'physiotherapist': physiotherapist,
        'diagnosis': diagnosis
    }
    patients.append(record)
    _save_data(PATIENTS_FILE, patients)
    print(f'Added patient ID {new_id}')

def view_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print('No patients found.')
        return
    for p in patients:
        print(f"ID: {p['id']}, Name: {p['name']}")

def search_patient():
    term = input('Search by name: ').strip().lower()
    patients = _load_data(PATIENTS_FILE)
    results = [p for p in patients if term in p['name'].lower()]
    if not results:
        print('No matches found.')
        return
    for p in results:
        print(f"ID: {p['id']}, Name: {p['name']}")

# SOAP note input

def add_soap_note():
    patient_id = select_patient('Enter patient name to add SOAP note: ')
    if patient_id is None:
        return
    notes = _load_data(NOTES_FILE)
    date_str = input('Session date (YYYY-MM-DD): ').strip()
    subj = input('Subjective: ').strip()
    obj = input('Objective: ').strip()
    action = input('Action: ').strip()
    plan = input('Plan: ').strip()
    entry = {
        'patient_id': patient_id,
        'date': date_str,
        'subjective': subj,
        'objective': obj,
        'action': action,
        'plan': plan
    }
    notes.append(entry)
    _save_data(NOTES_FILE, notes)
    print('SOAP note saved.')

# View SOAP notes

def view_soap_notes():
    patient_id = select_patient('Enter patient name to view notes: ')
    if patient_id is None:
        return
    notes = _load_data(NOTES_FILE)
    filtered = [n for n in notes if n['patient_id'] == patient_id]
    if not filtered:
        print('No notes for this patient.')
        return
    for n in sorted(filtered, key=lambda x: x['date']):
        print(f"Date: {n['date']}\n  S: {n['subjective']}\n  O: {n['objective']}\n  A: {n['action']}\n  P: {n['plan']}\n")

# Report generation

def generate_progress_report():
    if Document is None:
        print("Cannot generate report: python-docx missing.")
        return
    patient_id = select_patient('Enter patient name for report: ')
    if patient_id is None:
        return
    start = input('Start date (YYYY-MM-DD): ').strip()
    end = input('End date (YYYY-MM-DD): ').strip()
    patients = _load_data(PATIENTS_FILE)
    notes = _load_data(NOTES_FILE)
    patient = next((p for p in patients if p['id'] == patient_id), None)
    if not patient:
        print('Patient not found.')
        return
    entries = [n for n in notes
               if n['patient_id'] == patient_id
               and datetime.fromisoformat(start) <= datetime.fromisoformat(n['date']) <= datetime.fromisoformat(end)]
    if not entries:
        print('No notes in given range.')
        return
    entries.sort(key=lambda x: x['date'])
    doc = Document()
    style = doc.styles['Normal']
    style.font.name = 'Calibri'
    style.font.size = Pt(11)
    table = doc.add_table(rows=5, cols=2)
    row = table.rows[0].cells
    row[0].text = f"Patient Name: {patient.get('name','')}"
    row[1].text = f"Patient ID: {patient.get('id','')}"
    row = table.rows[1].cells
    row[0].text = f"Date of Birth: {patient.get('dob','')}"
    row[1].text = f"Date of Initial Assessment: {entries[0].get('date','')}"
    row = table.rows[2].cells
    row[0].text = f"Medical Aid: {patient.get('medical_aid','')}"
    row[1].text = f"Medical Aid Number: {patient.get('medical_aid_number','')}"
    row = table.rows[3].cells
    row[0].text = f"Occupation: {patient.get('occupation','')}"
    row[1].text = f"Physiotherapist: {patient.get('physiotherapist','')}"
    row = table.rows[4].cells
    report_date = datetime.now().date().isoformat()
    row[0].text = f"Report Date: {report_date}"
    row[1].text = f"Reason for Referral/Chief Complaint: {patient.get('diagnosis','')}"
    doc.add_paragraph()
    first = entries[0]
    para1 = f"{first.get('subjective','')} {first.get('objective','')}"
    doc.add_paragraph(para1)
    actions = [n.get('action','') for n in entries]
    doc.add_paragraph('. '.join(actions) + '.')
    plans = [n.get('plan','') for n in entries]
    doc.add_paragraph('. '.join(plans) + '.')
    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('Regards')
    doc.add_paragraph()
    doc.add_paragraph()
    doc.add_paragraph('______________________________')
    doc.add_paragraph('Ronelle Isaacs Physiotherapist')
    safe_name = patient.get('name','').replace(' ', '_') or 'Patient'
    filename = f"{safe_name}_Report_{start}_to_{end}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# CLI

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('View all patients', view_patients),
        '3': ('Search patients by name', search_patient),
        '4': ('Add SOAP note', add_soap_note),
        '5': ('View patient notes', view_soap_notes),
        '6': ('Generate progress report (Word)', generate_progress_report),
        '0': ('Exit', None)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for k, (desc, _) in options.items():
            print(f"{k}. {desc}")
        choice = input('Select an option: ').strip()
        if choice == '0':
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print('Invalid selection.')

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients by name
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  5
Enter patient name to view notes:  Michael


Selected: ID 1, Name: Michael
Date: 2025-04-22
  S: Got hit
  O: Looked bad
  A: Made it better
  P: Keep it better


=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients by name
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit


Select an option:  6
Enter patient name for report:  Michael


Selected: ID 1, Name: Michael


Start date (YYYY-MM-DD):  2025-04-01
End date (YYYY-MM-DD):  2025-04-30


Report saved to Michael_Report_2025-04-01_to_2025-04-30.docx

=== Physio EHR CLI ===
1. Add patient
2. View all patients
3. Search patients by name
4. Add SOAP note
5. View patient notes
6. Generate progress report (Word)
0. Exit
